In [13]:
import aiohttp
import asyncio
import json
import os
import settings
import requests

from requests.exceptions import HTTPError
from bimservice import get_ids
from aiohttp import ClientSession
from notion import Page, Filter

In [14]:
ssgbim_ids = get_ids('All')

In [15]:
def extract_fields_from_response(response):
    """Extract family data from bimservice"""
    ssgfid = response.get('familyId', '')
    family_name = response.get('familyName', '')
    # category_name = response.get('categoryName', '').split('/')[0]
    # object_type = response.get('familyObjectType')
    # status_index = response.get('status', 2)
    # if status_index == 0:
    #     status = 'Public'
    # elif status_index == 1:
    #     status = 'Private'
    # else:
    #     status = 'Work in Progress'

    return (ssgfid, family_name)

In [16]:
# Synchronous

for family_id in ssgbim_ids:
    url = settings.BIMSERVICE_BASE_URL + "Family/" + family_id
    # url = settings.GET_FAMILY + "Family/" + family_id
    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'An HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'An error occurred: {err}')
    response_json = response.json()
    extract_fields_from_response(response_json)

In [ ]:
async def query_for_id(family_id, session):
    """Query for notion page id (asynchronously)"""
    db_id = settings.DATABASE_IDS['Content Calendar']
    url = settings.NOTION_DATABASE_ENDPOINT + db_id + "/query"
    response = None
    filt = Filter.create_text_filter(family_id, database_property='SSGFID')
    try:
        response = await session.request(method='POST', url=url, data=json.dumps(filt), headers=settings.NOTION_HEADERS)
        response.raise_for_status()
        print(f'Response status ({url}): {response.status}')
    except HTTPError as http_err:
        print(f'An HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'An error occurred: {err}')
    response_json = await response.json()
    return response_json.get('id')


In [ ]:
async def post_family_data(page_id, session):
    """Post family data using bimservice (asynchronously)"""
    url = settings.NOTION_PAGE_ENDPOINT + page_id
    try:
        response = await session.request(method='POST', url=url, headers=settings.NOTION_HEADERS)
        response.raise_for_status()
        print(f'Response status ({url}): {response.status}')
    except HTTPError as http_err:
        print(f'An HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'An error occurred: {err}')
    response_json = await response.json()
    return response_json

In [9]:
async def get_family_data(family_id, session):
    """Get family data using bimservice (asynchronously)"""
    url = settings.BIMSERVICE_BASE_URL + "Family/" + family_id
    try:
        response = await session.request(method='GET', url=url, headers=settings.BIMSERVICE_HEADERS)
        response.raise_for_status()
        print(f'Response status ({url}): {response.status}')
    except HTTPError as http_err:
        print(f'An HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'An error occurred: {err}')
    response_json = await response.json()
    return response_json

In [18]:
async def main(family_id, session):
    """Wrapper for running program in an asynchronous manner"""
    try:
        response = await get_family_data(family_id, session)
        parsed_response = extract_fields_from_response(response)
        # print(f'Response: {json.dumps(parsed_response, indent=2)}')
        return json.dumps(parsed_response)
    except Exception as err:
        print(f'Exception occurred: {err}')
        pass

In [19]:
async with ClientSession() as session:
    await asyncio.gather(*[main(family_id, session) for family_id in ssgbim_ids])

Response status (https://bimservice.ssgbim.com/api/Family/f9efc791-9de7-420e-9882-03e8ca391f3c): 200
Response status (https://bimservice.ssgbim.com/api/Family/c5c05448-2d8d-423b-886a-00e2a188cfe7): 200
Response status (https://bimservice.ssgbim.com/api/Family/96c9dc50-b865-4bbf-8ebf-03cb6a47354c): 200
Response status (https://bimservice.ssgbim.com/api/Family/a5b31f73-043f-4af6-9777-04533b4550f6): 200
Response status (https://bimservice.ssgbim.com/api/Family/268c3950-47a3-468f-a575-01fa409dd894): 200
Response status (https://bimservice.ssgbim.com/api/Family/d4d0236f-5e55-4f39-a700-04550b6f06b4): 200
Response status (https://bimservice.ssgbim.com/api/Family/e0102243-2720-4c92-b49f-00fe4904c39d): 200
Response status (https://bimservice.ssgbim.com/api/Family/14931d48-5312-4cec-824a-024c5233a725): 200
Response status (https://bimservice.ssgbim.com/api/Family/8416e17f-4aff-45f5-a890-019801958b74): 200
Response status (https://bimservice.ssgbim.com/api/Family/94f6629f-a947-49f9-b9fe-003c78a1b